A simple jupyter widgets voltage/current selector button for the GS210

In [ ]:
# importing packages
import numpy as np
import time
import qcodes
import pyvisa as visa
import ipywidgets as widgets
# enable to get a more debuggable output:
#visa.log_to_screen()

# pyvisa-py resource manager
py_rm = visa.ResourceManager('@py')
# national instruments resource manager
ni_rm = visa.ResourceManager()

/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/gpib_ctypes/gpib/gpib.py:54: UserWarning: GPIB library not found. Please manually load it using _load_lib(filename). All GPIB functions will raise OSError until the library is manually loaded.
  warnings.warn(message)


In [2]:
# show the interfaces that the visapy-py backend detects
py_rm_interface = widgets.Dropdown(
    options=py_rm.list_resources(),
    #value='2',
    description='py Interface:',
    disabled=False,
    layout={'width': 'max-content'}
)

display(py_rm_interface)

Dropdown(description='py Interface:', layout=Layout(width='max-content'), options=('ASRL/dev/ttyS4::INSTR', 'U…

In [3]:
### IMPORTANT ###
# for some reason, the USB interface of the instrument has to be opened with the pyvisa-py resource manager before ni-visa

# so, open whichever interface was selected in the prior cell
py_rm.open_resource(py_rm_interface.value)

# show the interfaces that the pyvisa-py backend detects
ni_rm_interface = widgets.Dropdown(
    options=ni_rm.list_resources(),
    #value='2',
    description='ni Interface:',
    disabled=False,
    layout={'width': 'max-content'}
)

display(ni_rm_interface)

Dropdown(description='ni Interface:', layout=Layout(width='max-content'), options=('USB0::0x0B21::0x0039::91LA…

In [4]:
# importing an instrument driver
from qcodes.instrument_drivers.yokogawa.GS200 import GS200

# intialize the instrument with the following parameters

# the instrument is initialized and stored as a variable gs, where "gs200" is its name
gs = GS200("gs200", address=ni_rm_interface.value, terminator="\n")

# close the pyvisa-py resource manager:
py_rm.close()

Connected to: YOKOGAWA GS210 (serial:91LA25023, firmware:1.05) in 0.25s


In [5]:
# resetting the instrument
gs.reset()
gs.auto_range(True)

In [6]:
# code for the toggle buttons to work with the GS210
import ipywidgets as widgets
from IPython.display import display
mode_toggle = widgets.ToggleButtons(
    options=['Voltage', 'Current'],
    description='Mode:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

def changed(change):
    print(mode_toggle.value)
    gs.source_mode(mode_toggle.value[:4].upper())
    
mode_toggle.observe(changed, 'value')

mode_toggle

ToggleButtons(description='Mode:', options=('Voltage', 'Current'), value='Voltage')

In [7]:
# close the interface
gs.close()